In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

X_train = pd.read_csv("dataset/train_features.csv")
y_train = pd.read_csv("dataset/train_labels.csv")
X_test = pd.read_csv("dataset/valid_features.csv")
y_test = pd.read_csv("dataset/valid_labels.csv")

X_train = pd.DataFrame(X_train, columns=["loudness","title","tempo", "time_signature","key","mode","duration"])
X_test = pd.DataFrame(X_test, columns=["loudness","title","tempo", "time_signature","key","mode","duration"])

cleanup_nums = {"genre": 
                {"classic pop and rock": 1, 
                 "dance and electronica": 2,
                 "folk":3,
                 "jazz and blues":4,
                 "metal":5,
                 "pop":6,
                 "punk":7,
                 "soul and reggae":8
                }}
y_train.replace(cleanup_nums, inplace=True)
y_test.replace(cleanup_nums, inplace=True)


tfidf_vect = TfidfVectorizer()
X_title_cv_train = tfidf_vect.fit_transform(X_train['title'])
df_title_cv_train = pd.DataFrame(X_title_cv_train.toarray(), columns=tfidf_vect.get_feature_names())
X_train = pd.concat([X_train, df_title_cv_train], axis=1)
X_train = X_train.drop(["title"], 1)

X_title_cv_test = tfidf_vect.transform(X_test['title'])
df_title_cv_test = pd.DataFrame(X_title_cv_test.toarray(), columns=tfidf_vect.get_feature_names())
X_test = pd.concat([X_test, df_title_cv_test], axis=1)
X_test = X_test.drop(["title"], 1)

#scaler = MinMaxScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)
X_train['time_signature'].unique()

array([4, 3, 1, 7, 5, 0])

In [4]:
knn = KNeighborsClassifier(n_neighbors=10)
y_train["genre"] = y_train["genre"].astype('int')
knn.fit(X_train, y_train["genre"])
preds=knn.predict(X_test)

# let's how our model performed
print("Classificaion report: \n")
print(classification_report(y_test["genre"], preds))
print("Confusion matrix: \n")
print(confusion_matrix(y_test["genre"], preds))

print("\n\n\n")

def evaluate(pred, true):
    CM = metrics.confusion_matrix(true, pred) # Confusion Matrix
    Acc = metrics.accuracy_score(true, pred) # Accuracy
    precf1 = metrics.precision_recall_fscore_support(true, pred) # Precision, Recall and F1-score
    return CM, Acc, precf1

CM, Acc, precf1 = evaluate(preds, y_test["genre"])

correct = 0
for i in range(len(y_test)):
    correct = correct + int(preds[i]==y_test["genre"][i])
    
print("Confusion Matrix:\n{}\naccuracy: {}\naccuracy by sklearn.metric: {}\nprecision: {}\nrecall: {}\nF1: {}\n".format(
                                                CM,
                                                correct / len(y_test), 
                                                Acc,
                                                precf1[0],
                                                precf1[1],
                                                precf1[2]))

Classificaion report: 

              precision    recall  f1-score   support

           1       0.15      0.75      0.25        55
           2       0.19      0.11      0.14        45
           3       0.40      0.19      0.26        64
           4       0.00      0.00      0.00        44
           5       0.59      0.15      0.24        66
           6       0.83      0.34      0.48        74
           7       0.17      0.11      0.14        44
           8       0.25      0.17      0.20        58

    accuracy                           0.24       450
   macro avg       0.32      0.23      0.21       450
weighted avg       0.37      0.24      0.23       450

Confusion matrix: 

[[41  3  4  0  1  1  1  4]
 [30  5  1  0  2  0  1  6]
 [40  3 12  0  1  0  1  7]
 [29  3  5  0  0  0  1  6]
 [40  3  4  0 10  0  7  2]
 [36  2  1  0  0 25  7  3]
 [29  2  2  0  0  4  5  2]
 [31  6  1  0  3  0  7 10]]




Confusion Matrix:
[[41  3  4  0  1  1  1  4]
 [30  5  1  0  2  0  1  6]
 [40  3 12  

/home/wildan/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
